Определение языка (language detection)
--------------------

* **Множество случаев** — тексты на разных языках
* **Множество классов** — языки

Я решила взять 4 языка, не очень связанных друг с другом. Это монгольский, польский, русинский и немецкий.
Первым делом выкачиваем статьи.

In [1]:
langues = ['mn', 'pl', 'rue', 'de']

In [2]:
def get_texts_for_lang(lang, n=10): # функция для скачивания статей из википедии
    wikipedia.set_lang(lang)
    wiki_content = []
    pages = wikipedia.random(n)
    for page_name in pages:
        try:
            page = wikipedia.page(page_name)
        except wikipedia.exceptions.WikipediaException:
            print('Skipping page {}'.format(page_name))
            continue

        wiki_content.append('{}\n{}'.format(page.title, page.content.replace('==', '')))

    return wiki_content

In [3]:
import wikipedia # скачиваем по 100 статей для каждого языка. Это может занять какое-то время (5-10 минут. как правило)

wiki_texts = {}
for lang in langues: # казахский в википедии — это kk,
                                      # украинский — uk, а белорусский — be
    wiki_texts[lang] = get_texts_for_lang(lang, 100)
    print(lang, len(wiki_texts[lang]))

C:\Users\1\Anaconda3\lib\site-packages\bs4\__init__.py:181: UserWarning: No parser was explicitly specified, so I'm using the best available HTML parser for this system ("lxml"). This usually isn't a problem, but if you run this code on another system, or in a different virtual environment, it may use a different parser and behave differently.

The code that caused this warning is on line 193 of the file C:\Users\1\Anaconda3\lib\runpy.py. To get rid of this warning, change code that looks like this:

 BeautifulSoup(YOUR_MARKUP})

to this:

 BeautifulSoup(YOUR_MARKUP, "lxml")

  markup_type=markup_type))


Skipping page Араб
Skipping page Анадырь (салаа утга)
mn 98
Skipping page Herat
Skipping page Sing-Sing (album)
Skipping page Pantone
Skipping page Rezonans magnetyczny
Skipping page Zarośla (Dąbrowa)
Skipping page Thomas Hicks
pl 94
rue 100
Skipping page Tom Kelly Band
Skipping page Emden (Begriffsklärung)
Skipping page Tarnow
Skipping page Partridge
Skipping page Wauchope
Skipping page Ardenne
Skipping page Kurt Runge (Ruderer)
Skipping page Hans Häberlin
de 92


Проверяем, скачались ли тексты.

In [5]:
print(wiki_texts['rue'][0])
print(wiki_texts['de'][0])

Лихтенштайн
Лихтенштайн (нім. Fürstentum Liechtenstein) — країна (конштітучна монархія) лежача в западній Европі. Лихтенштайн гранічіть з Австріёв, з Швейцаріёв. Головне місто є Вадуц. Популація — 36 476 (31.12.2011) жытелїв. Площа 160 км²;


 Названя 


 Ґеоґрафія 


 Адміністратівне дїлїня 


 Населїня 


 Історія 
Noah Saavedra
Noah Saavedra (* 13. Februar 1991 in Oberpullendorf, Burgenland) ist ein österreichischer Filmschauspieler.


 Leben 
Noah Saavedra hat chilenische Wurzeln; sein Großvater war wegen des Militärregimes unter Augusto Pinochet von Chile nach Österreich geflohen. Noah Saavedra spricht daher auch fließend Spanisch.
Nach dem Pflichtschulabschluss sammelte er von 2012 bis 2013 am Burgtheater in Wien erste Schauspielerfahrungen. Danach studierte er von 2013 bis 2015 Schauspiel am Konservatorium der Stadt Wien. Seit Herbst 2015 ist Saavedra Student an der Hochschule für Schauspielkunst „Ernst Busch“ Berlin eingeschrieben.
Seine erste Filmrolle übernahm er 2015 im Jame

### Первый метод: частотные слова

In [6]:
import collections
import codecs
import sys
import re

In [7]:
from string import punctuation, digits
punctuation = set(punctuation + '«»—…“”\n\t' + digits)

С очисткой у меня возникли небольшие проблемы. У знаков отсутствовала какая-либо закономерность, поэтому многие из них пришлось удалять отдельно, чтобы уж наверняка. 
Очистку вынесла в функцию, так как ниже использовала её отдельно от токенизации.

In [8]:
def clean(text):
    text = re.sub(r'\.| - =|,|; !', ' ', text)
    text = re.sub(r'=|·', ' ', text)
    text = re.sub(r';', ' ', text)
    text = re.sub(r'\+|/', ' ', text)
    text = re.sub(r'’|\'', ' ', text)
    text = re.sub('(.*?)-|:(.*?)', ' ', text)
    text = re.sub(r'\d|  |\[|\]|\(|\)|\\|—|\"|\„|\n|&|»|«', '', text)
    return text

In [9]:
def tokenize(text):
    text = clean(text)
    return text.split(' ')

Функция для собирания частотных слов. Возвращает сет.

In [10]:
def get_freq(wiki_texts, lang):
    freqs = collections.defaultdict(lambda: 0)
    corpus = wiki_texts[lang]
    for article in corpus:
        for word in tokenize(article.replace('\n', '').lower()):
            freqs[word] += 1
    return set(freqs)

In [11]:
freq_de = get_freq(wiki_texts, 'de')
freq_rue = get_freq(wiki_texts, 'rue')
freq_pl = get_freq(wiki_texts, 'pl')
freq_mn = get_freq(wiki_texts, 'mn')

Очищаем сеты от повторений и кладём их в словарь формата {'язык':'частотные слова'}.

In [12]:
dict_freq = {}
dict_freq['de'] = freq_de - freq_rue - freq_pl - freq_mn
dict_freq['rue'] = freq_rue - freq_de - freq_pl - freq_mn
dict_freq['pl'] = freq_pl - freq_de - freq_rue - freq_mn
dict_freq['mn'] = freq_mn - freq_de - freq_rue - freq_pl

Функция по определению языка на основе частотных слов.

In [13]:
def predict_language_f(text, dict_freq, langues):
    text_chars = tokenize(text.lower())
    lang2sim = {}
    
    for lang in langues:
        intersect = len(set(text_chars) & dict_freq[lang])
        lang2sim[lang] = intersect
    
    return max(lang2sim, key=lambda x: lang2sim[x])

### Второй метод: частотные символьные n-граммы

Функция, которая преобразовывает строку в массив n-грамм заданной длины.

In [14]:
from itertools import islice, tee

def make_ngrams(text):
    N = 3 # задаем длину n-граммы
    text = clean(text)
    text = text.strip()
    ngrams = zip(*(islice(seq, index, None) for index, seq in enumerate(tee(text, N))))
    ngrams_fin = [''.join(x) for x in ngrams]
    return ngrams_fin

Функция для собирания частотных n-грамм. Возвращает сет.

In [15]:
def get_ngrams(wiki_texts, lang):
    ngrams_l= collections.defaultdict(lambda: 0)
    corpus = wiki_texts[lang]
    for article in corpus:
        for ngram in make_ngrams(article.replace('\n', '').lower()):
            ngrams_l[ngram] += 1
    return set(ngrams_l.keys())

In [16]:
ngrams_fr = get_ngrams(wiki_texts, 'de')
ngrams_uk = get_ngrams(wiki_texts, 'rue')
ngrams_be = get_ngrams(wiki_texts, 'pl')
ngrams_kk = get_ngrams(wiki_texts, 'mn')

Очищаем сеты от повторений и кладём в словарь формата {'язык':'n-граммы'}

In [17]:
dict_ngrams = {}
dict_ngrams['de'] = ngrams_de - ngrams_rue - ngrams_pl - ngrams_mn
dict_ngrams['rue'] = ngrams_rue - ngrams_de - ngrams_pl - ngrams_mn
dict_ngrams['pl'] = ngrams_pl - ngrams_de - ngrams_rue - ngrams_mn
dict_ngrams['mn'] = ngrams_mn - ngrams_de - ngrams_rue - ngrams_pl

Функция по определению языка на основе частотных слов.

In [18]:
def predict_language_n(text, dict_ngrams, langues):
    text_chars = make_ngrams(text.lower())
    lang2sim = {}
    
    for lang in langues:
        intersect = len(set(text_chars) & dict_ngrams[lang])
        lang2sim[lang] = intersect
    
    return max(lang2sim, key=lambda x: lang2sim[x])

### Определяем язык

In [19]:
def predict_lang(wiki_texts, langues, dict_freq, dict_ngrams):
    check_n = []
    check_f = []
    for lang in langues:
        for texts in wiki_texts[lang]:
            c_f = predict_language_f(texts, dict_freq, langues)
            check_f.append(c_f == lang)
            c_n = predict_language_n(texts, dict_ngrams, langues)
            check_n.append(c_n == lang)
            
    return check_f, check_n

In [20]:
pred_freq, pred_ngram = predict_lang(wiki_texts, langues, dict_freq, dict_ngrams)

In [21]:
False in pred_freq

False

In [22]:
False in ngram

False

Оба метода определяют язык без ошибки.